In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import datetime
import re
import time
import os
import json
from PIL import Image 
import PIL 
pd.set_option('display.max_columns', None)
import random
import codecs
import subprocess
from selenium import webdriver
from bs4 import BeautifulSoup
import chromedriver_binary
from selenium.webdriver.chrome.options import Options

# Headers

In [2]:
headers = {
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'sec-ch-ua-mobile': '?0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
}

# Save pages

In [3]:
def save_page_list(req, page=1):
    datetime_1 = datetime.datetime.now().strftime("%Y-%m-%d_%Hh%M")
    page_list_name = "fulloccaz" + "-" + datetime_1 + "-" + str(page)
    with open("pages/" + page_list_name + ".html", "w")  as file:
        file.write(req.text)
        file.close()
    return "pages/" + page_list_name + ".html"

In [5]:
#def get_source_selenium(url_):
#    options = Options()
#    options.add_argument("--headless")
#    driver = webdriver.Chrome(options=options, executable_path="/Users/victor/Downloads/chromedriver")
#    driver.get(url_)
#    page_source = driver.page_source
#    driver.quit()
#    return page_source

In [ ]:
scrap = 1
page_=1
#
while scrap==1:
    html_file = requests.get(f"https://www.fulloccaz.com/?&page={page_}", headers=headers)
    last_saved_file = save_page_list(html_file, page=page_)
    
    with open(last_saved_file, 'r') as f:
        readable_html = f.read()
        prices = get_prices_from_pages(readable_html)
    if len(prices) == 0:
        scrap=0

    page_ += 1    
    time.sleep(random.randint(1, 3))
    print("page saved: " + str(page_))

# Extract data from pages

## Test file

In [4]:
file_test_name = [file for file in os.listdir("pages") if file.endswith("html")][-2]
file_test_name

'fulloccaz-2021-06-05_18h16-27.html'

In [5]:
file_check_x15_good = f"pages/{file_test_name}"
file_check_x15_good

'pages/fulloccaz-2021-06-05_18h16-27.html'

## Get prices from pages

In [6]:
def get_prices_from_pages(r):
    
    # set empty price list
    price_list = []
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract price
    step1 = soup.select('div[id*="IDVignettes"]')
    if len(step1) > 0:
        step2 = step1[0].select('div[class*="CLPrix"]')
        if len(step2) > 0:
            for k in step2:
                try:
                    price_list.append(float("".join(re.findall("[0-9]", k.text.replace("\n", "").replace("\t", "")))))
                except:
                    price_list.append(np.nan)    
    return price_list

In [7]:
with open(file_check_x15_good, 'r') as f:
    readable_html = f.read()
    prices = get_prices_from_pages(readable_html)
prices

[6490.0,
 9590.0,
 5290.0,
 10790.0,
 13590.0,
 1290.0,
 7990.0,
 4500.0,
 3490.0,
 8999.0,
 1350.0,
 5590.0]

## Get url from pages

In [8]:
def get_urls_from_pages(r):
    
    # set empty url list
    url_list = []
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract price
    step1 = soup.select('div[id*="IDVignettes"]')
    if len(step1) > 0:
        step2 = step1[0].select('a')
        if len(step2) > 0:
            for k in step2:
                if "href" in k.attrs:
                    try:
                        url_list.append("https://www.fulloccaz.com" + k.attrs["href"])
                    except:
                        url_list.append(np.nan)    
    return url_list

In [9]:
with open(file_check_x15_good, 'r') as f:
    readable_html = f.read()
    urls = get_urls_from_pages(readable_html)
urls

['https://www.fulloccaz.com/fiches/index/2028161-occasion-yamaha-fjr-1300a?',
 'https://www.fulloccaz.com/fiches/index/2015502-occasion-kawasaki-versys-650-se-gt?',
 'https://www.fulloccaz.com/fiches/index/2033933-occasion-kymco-400-xciting-s?',
 'https://www.fulloccaz.com/fiches/index/2049541-occasion-triumph-street-triple-765-rs?',
 'https://www.fulloccaz.com/fiches/index/2033378-occasion-yamaha-niken-850?',
 'https://www.fulloccaz.com/fiches/index/2030313-occasion-kymco-zing-125-zing125?',
 'https://www.fulloccaz.com/fiches/index/2049506-occasion-kymco-ak-550-abs-e4-7990?',
 'https://www.fulloccaz.com/fiches/index/2048061-occasion-300-mp3?',
 'https://www.fulloccaz.com/fiches/index/2040258-occasion-honda-cbf-1000-f?',
 'https://www.fulloccaz.com/fiches/index/2049473-occasion-yamaha-tracer-900-gt?',
 'https://www.fulloccaz.com/fiches/index/2035911-occasion-yamaha-majesty-125?',
 'https://www.fulloccaz.com/fiches/index/2016668-occasion-kymco-xciting-400i-s?']

## Get references from pages

In [10]:
def get_references_from_url_list(url_list_):
    
    # set empty ref list
    ref_list = []
    
    for k in url_list_:
        try:
            ref_list.append(k.split("/")[5].split("-")[0])
        except:
            ref_list.append(np.nan)
    #                
    return ref_list

In [11]:
get_references_from_url_list(urls)

['2028161',
 '2015502',
 '2033933',
 '2049541',
 '2033378',
 '2030313',
 '2049506',
 '2048061',
 '2040258',
 '2049473',
 '2035911',
 '2016668']

# Download single announce pages

In [12]:
data_fulloccaz = "../../../tresboncoin/data/fulloccaz.csv"
#data = pd.read_csv(data_motoplanete)

In [13]:
def add_the_announce(df, uniq_id, price):
    if int(uniq_id) in list(df["unique id"]):
        index_ = df.index[df['unique id'] == int(uniq_id)].tolist()[0]
        
        # if an announce with same uniq id and same price is found, return true and skip
        return df.iloc[index_]["price"]!=price
    
    # else, return false and add the announce
    return True

In [14]:
def save_page_uniq(req, uniq_id):
    datetime_1 = datetime.datetime.now().strftime("%Y-%m-%d_%Hh%M")
    page_name = "fulloccaz" + "-" + uniq_id + "-" + datetime_1
    with open("annonces/" + page_name + ".html", "w")  as file:
        file.write(req.text)
        file.close()

In [15]:
def file_exists(uniq_id):
    for k in os.listdir("annonces"):
        if "-" + str(uniq_id) + "-" in k:
            return True
    return False

In [304]:
#qq = "https://www.fulloccaz.com/fiches/index/2016930-occasion-piaggio-mp3-300-hpe-abs-asr?"
#req_uniq = requests.get(qq, headers = headers)
#save_page_uniq(req_uniq, "2016930")

In [ ]:
announce_count = 1
page_count = 0
for html_file in [file for file in os.listdir("pages") if file.endswith("html")]:
    page_count +=1
    print("page n°: " + str(page_count))
    with open(f"pages/{html_file}", 'r') as f:
        readable_html = f.read()
        urls_list = get_urls_from_pages(readable_html)
        unique_id = get_references_from_url_list(urls_list)
        prices_list = get_prices_from_pages(readable_html)
    if urls_list != None:
        for url_single, uniq_id, price_ in zip(urls_list, unique_id, prices_list):
            
            #if add_the_announce(data_price, uniq_id, price_):
            #html_file = get_source_selenium(url_single)
            if file_exists(uniq_id) == False:
                req_uniq = requests.get(url_single, headers = headers)
                save_page_uniq(req_uniq, uniq_id)
                print(str(page_count) + ". saved page id: " + uniq_id)
                time.sleep(random.randint(1, 5))
                announce_count += 1
            else:
                print(announce_count, url_single + " --> skipped")
                announce_count += 1

# Process single announces
Saving scaled images x3 and adding announce to DataFrame

In [438]:
announce_template = pd.DataFrame({"url": [np.nan],
                                  "unique id": [np.nan],
                                  "date_scrapped": [np.nan],
                                  "announce_publication_date": [np.nan],
                                  "vehicle brand": [np.nan],
                                  "vehicle type": [np.nan],
                                  "color": [np.nan],
                                  "price": [np.nan],
                                  "city": [np.nan],
                                  "postal code": [np.nan],
                                  "seller": [np.nan],
                                  "seller_name": [np.nan],
                                  "vehicle release date": [np.nan],
                                  "mileage": [np.nan],
                                  "motorisation": [np.nan],
                                  "fiscal power": [np.nan],
                                  "guarantee": [np.nan],
                                  "hand": [np.nan],
                                  "chassis": [np.nan],
                                  "engine capacity [CC]": [np.nan],
                                  "comments": [np.nan]})
announce_template

,url,unique id,date_scrapped,announce_publication_date,vehicle brand,vehicle type,color,price,city,postal code,seller,seller_name,vehicle release date,mileage,motorisation,fiscal power,guarantee,hand,chassis,engine capacity [CC],comments
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## test announce

In [439]:
file_name = os.listdir("annonces")[-1]
#
file_check_single = f"annonces/{file_name}"
file_check_single

'annonces/fulloccaz-2015545-2021-06-06_12h24.html'

## Functions

### Get url

In [440]:
def get_url(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract url
    step1 = soup.select('form[class*="CLFboxForm"]')
    if len(step1)>0:
        try:
            return step1[0].attrs["pagesite"]
        except:
            return np.nan
    return np.nan

In [441]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_url = get_url(readable_html)
g_url

'https://www.fulloccaz.com/fiches/index/2015545-occasion-bmw-r1200rt'

### Get unique id

In [491]:
def get_uniq_id(url_):
    try:
        return url_.split("/")[-1].split("-")[0]
    except:
        return np.nan

In [492]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_url = get_url(readable_html)
get_uniq_id(g_url)

'2015545'

### get publication date

In [444]:
def get_publication_date(r):

    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Mise à jour") >= 0:
                    try:
                        return k.select('td')[1].text
                    except:
                        return np.nan
    else:
        return np.nan

In [445]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_pub = get_publication_date(readable_html)
g_pub

'05/06/2021'

### Get vehicule brand

In [446]:
def get_brand_and_model(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('h1')
    if len(step1)>0:
        return step1[0].text
    else:
        return None

In [447]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_brand = get_brand_and_model(readable_html)
g_brand

'BMW R1200RT'

### Get vehicule type

In [448]:
def get_type(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Catégorie") >= 0:
                    try:
                        return k.select('td')[1].text
                    except:
                        return np.nan
    else:
        return np.nan

In [449]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_type = get_type(readable_html)
g_type

'Routière'

### get color

In [450]:
def get_color(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    
    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Couleur") >= 0:
                    try:
                        return k.select('td')[1].text
                    except:
                        return np.nan
    else:
        return np.nan

In [451]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_type = get_color(readable_html)
g_type

### get price

In [452]:
def get_price(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')

    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Prix") >= 0:
                    try:
                        return float("".join(re.findall("\d+", k.select('td')[1].text)))
                    except:
                        return np.nan
    else:
        return np.nan

In [453]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_type = get_price(readable_html)
g_type

5990.0

### get city

In [454]:
def get_city(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLAnnonceConcession"]')
    if len(step1)>0:
        try:
            check_paris = step1[0].select('div[id*="IDadresse"]')[0].text.split("\n")[-1].replace("\t", " ").strip().split(" ")
            if len(re.findall("\d+", check_paris[-1])) > 0:
                return check_paris[-2]
            else:
                return check_paris[-1]
        except:
            return np.nan
    return None

In [455]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_city = get_city(readable_html)
g_city

'ORVAULT'

### get postal code

In [456]:
def get_postal(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLAnnonceConcession"]')
    if len(step1)>0:
        try:
            splited_adress = step1[0].select('div[id*="IDadresse"]')[0].text.split("\n")[-1].replace("\t", " ").strip().split(" ")
            for k in splited_adress:
                find_post_code = re.findall("[0-9]{5}", k)
                if len(find_post_code) == 1:
                    return find_post_code[0]
        except:
            pass
    return np.nan

In [457]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_post = get_postal(readable_html)
g_post

'44700'

### get seller

In [458]:
def get_seller(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLAnnonceConcession"]')
    if len(step1)>0:
        try:
            step2 = step1[0].select("strong")[0].text.split(":")
            return step2[-1].strip()
        except:
            pass
    return np.nan

In [459]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_seller = get_seller(readable_html)
g_seller

'NANTES MOTO'

### get seller name

In [460]:
def get_seller_name(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLAnnonceConcession"]')
    if len(step1)>0:
        try:
            step2 = step1[0].select('div[id*="IDadresse"]')
            step3 = step2[0].text.replace("\t", " ").split("\n")
            for k in step3:
                if k.find("Votre contact") >= 0:
                    return k.split(":")[-1].strip()
        except:
            pass
    return np.nan

In [461]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_seller_name = get_seller_name(readable_html)
g_seller_name

'Un commercial, Steven ou David'

### get vehicle release date

In [462]:
def get_release(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Date de mise") >= 0:
                    try:
                        return k.select('td')[1].text
                    except:
                        return np.nan
    else:
        return np.nan

In [463]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_rel = get_release(readable_html)
g_rel

'04/08/2009'

### get vehicle mileage

In [464]:
def get_mileage(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Kilom") >= 0:
                    try:
                        return float(re.findall("\d+", k.select('td')[1].text)[0])
                    except:
                        return np.nan
    else:
        return np.nan

In [465]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_mileage = get_mileage(readable_html)
g_mileage

103000.0

### get engine capacity

In [466]:
def get_capa(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Cylind") >= 0:
                    try:
                        return float(re.findall("\d+", k.select('td')[1].text)[0])
                    except:
                        return np.nan
    else:
        return np.nan

In [467]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_capa = get_capa(readable_html)
g_capa

1200.0

### get engine type

In [468]:
def get_engine_type(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Motori") >= 0:
                    try:
                        return k.select('td')[1].text
                    except:
                        return np.nan
    else:
        return np.nan

In [469]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_et = get_engine_type(readable_html)
g_et

'4T'

### get fiscal power

In [470]:
def get_fiscal_power(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Puissance") >= 0:
                    try:
                        return k.select('td')[1].text
                    except:
                        return np.nan
    else:
        return np.nan

In [471]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_fp = get_fiscal_power(readable_html)
g_fp

### get guarantee

In [472]:
def get_guarantee(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Garant") >= 0:
                    try:
                        return k.select('td')[1].text.replace("\n", "").replace("\t", "").strip()
                    except:
                        return np.nan
    else:
        return np.nan

In [473]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_guar = get_guarantee(readable_html)
g_guar

'6 mois'

### get hand

In [474]:
def get_hand(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Première") >= 0:
                    try:
                        return "First hand"
                    except:
                        return np.nan
    else:
        return np.nan

In [475]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_hand = get_hand(readable_html)
g_hand

### get chassis

In [476]:
def get_chassis(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLInfosAnnonces"]')
    if len(step1)>0:
        step2 = step1[0].select('tr')
        if len(step2) > 0:
            for k in step2:
                if k.select('td')[0].text.find("Chassis") >= 0:
                    try:
                        return k.select('td')[1].text
                    except:
                        return np.nan
    else:
        return np.nan

In [477]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_chassis = get_chassis(readable_html)
g_chassis

### get comment

In [478]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', raw_html)
    return cleantext

In [479]:
def get_comment(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="CLAnnonceDescription"]')
    if len(step1)>0:
        try:
            return cleanhtml(step1[0].text.replace("\n", " ").replace("\t", " ").replace("  ", " ").strip())
        except:
            return ""
    return ""

In [480]:
with codecs.open(file_check_single, 'r', encoding='utf-8', errors='ignore') as f:
    readable_html = f.read()
    g_comment = get_comment(readable_html)
g_comment

'Description Equipements de série : ABS, Poignées chauffantes, Prise 12V, Top case 2 casques, Valises, Version GTEquipements optionnels inclus : Silencieux AkrapovicRemarques entretien : Aucun consommable à prévoir, Factures d‘entretien, Révision faiteServices concession : Démarches administratives sur place, Essai sur rendez-vous, Financement & Assurance possibles'

### Save image functions

In [481]:
def get_images(r, uniq_id):
    
    # get images url list
    image_list = []
    
    # get soup
    soup_ = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup_.select('div[class*="CLSlide-group"]')
    
    if len(step1)>0:
        step2 = step1[0].select("a")
        if len(step2) > 0:
            for k in step2:
                if "href" in k.attrs:
                    image_list.append(k.attrs['href'])

    # save images
    k=1
    for image_url in image_list[0:3]:
        image_name = f'images/{uniq_id}-{k}.jpg'
        if os.path.isfile(image_name) is False:
            img_data = requests.get(image_url).content
            with open(image_name, 'wb') as handler:
                handler.write(img_data)
            try:
                image = Image.open(image_name) 
                ratio = image.size[0] / image.size[1]
                image = image.resize((300,int(300/ratio)))
                image.save(f'images/{uniq_id}-{k}.jpg',optimize = True, quality = 70)
            except:
                pass
        k+=1
        
    return

In [431]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_url = get_url(readable_html)
    get_images(readable_html, get_uniq_id(g_url))

https://www.fulloccaz.com/nas/commun/annonces//a6b9fff993692e166f34a41bd2eea0ef8c8b833d/photo-annonce-kymco-ak-550_5f4e6f385481d443057694.jpg
https://www.fulloccaz.com/nas/commun/annonces//a6b9fff993692e166f34a41bd2eea0ef8c8b833d/photo-annonce-kymco-ak-550_5f4e6f3d8028a305583159.jpg
https://www.fulloccaz.com/nas/commun/annonces//a6b9fff993692e166f34a41bd2eea0ef8c8b833d/photo-annonce-kymco-ak-550_5f4e6f42683ca632103229.jpg


<div style="color:orangered; font-weight:700; font-size:20pt">TEST 100 ANNONCES</div>

In [ ]:
counter = 1
list_test = []
for single_ in os.listdir("annonces"):
    with open(f"annonces/{single_}", 'r') as f:
        readable_html = f.read()
        g_X = get_comment(readable_html)
        print(single_, counter, g_X)
        counter+=1

## Extract 1 announce

In [485]:
def process_announce(file_name):
    """ this function add an announce content from fulloccaz.com to the dataframe """
    
    # load csv if exists or starting from template
    data_fulloccaz = "../../../tresboncoin/data/fulloccaz.csv"
    #
    if os.path.isfile(data_fulloccaz) is False:
        pass
    else:
        data = pd.read_csv(data_fulloccaz)
    
    df = announce_template.copy()
    
    # fill df with defined functions
    with open(f"annonces/{file_name}", 'r') as f:
        readable_html = f.read()
        
        df["url"] = get_url(readable_html)
        df["unique id"] = get_uniq_id(get_url(readable_html))
        df["date_scrapped"] = datetime.datetime.now().strftime("%Y/%m/%d - %Hh%M")
        df["announce_publication_date"] = get_publication_date(readable_html)
        df["vehicle brand"] = get_brand_and_model(readable_html)
        df["vehicle type"] = get_type(readable_html)
        df["color"] = get_color(readable_html)
        df["price"] = get_price(readable_html)
        df["city"] = get_city(readable_html)
        df["postal code"] = get_postal(readable_html)
        df["seller"] = get_seller(readable_html)
        df["seller_name"] = get_seller_name(readable_html)
        df["vehicle release date"] = get_release(readable_html)
        df["mileage"] = get_mileage(readable_html)
        df["motorisation"] = get_engine_type(readable_html)
        df["fiscal power"] = get_fiscal_power(readable_html)
        df["guarantee"] = get_guarantee(readable_html)
        df["hand"] = get_hand(readable_html)
        df["chassis"] = get_chassis(readable_html)
        df["engine capacity [CC]"] = get_capa(readable_html)
        df["comments"] = get_comment(readable_html)
    
        # save images
        get_images(readable_html, get_uniq_id(get_url(readable_html)))
        
    # concatenate to csv and write
    try:
        data = pd.concat([data, df], axis=0)
        data.to_csv(path_or_buf = data_fulloccaz, index=False)
    except:
        df.to_csv(path_or_buf = data_fulloccaz, index=False)
    
    # deplacer le fichier html traité dans le vault
    subprocess.run(["mv", f"annonces/{file_name}", "vault"])
    return

In [486]:
list(os.listdir("annonces")[0:2])

['fulloccaz-2011022-2021-06-06_13h06.html',
 'fulloccaz-1932135-2021-06-06_12h33.html']

In [493]:
for single_ in [file for file in os.listdir("annonces") if file.endswith(".html")]:
    process_announce(single_)

In [489]:
data_moto = "../../../tresboncoin/data/fulloccaz.csv"
data = pd.read_csv(data_moto)
data.tail(20)

,url,unique id,date_scrapped,announce_publication_date,vehicle brand,vehicle type,color,price,city,postal code,seller,seller_name,vehicle release date,mileage,motorisation,fiscal power,guarantee,hand,chassis,engine capacity [CC],comments
13,https://www.fulloccaz.com/fiches/index/2012739...,2012739,2021/06/06 - 14h16,18/05/2021,HONDA GL1800 GOLDWING,Routière,Beige,11990.0,TETEGHEM,59229,WANTED BIKE,Fred,06/06/2003,31800.0,4T,16 CV,Non Garantie,NaN,NaN,1800.0,Description HONDA GL1800 GOLDWING DU 06/2003...
14,https://www.fulloccaz.com/fiches/index/2013344...,2013344,2021/06/06 - 14h16,25/05/2021,YAMAHA YZF-R125,Sportive,Bleu,4999.0,MONTLUCON,3100,MOTORAMA,Arthur ou Daniel,25/05/2021,1.0,4T,1 CV,24 mois,NaN,NaN,125.0,Description YAMAHA YZF 125 R 2018véhicule ne...
15,https://www.fulloccaz.com/fiches/index/2029016...,2029016,2021/06/06 - 14h16,12/04/2021,RIEJU TANGO 125 COLORIS NOIR / ROUGE,Moto 125,Noir,3299.0,HAGUENAU,67500,CK MOTOS,Christophe,31/12/2020,NaN,Essence,NaN,24 mois jusqu'au 01/01/1970,NaN,NaN,125.0,Description A SAISIR RIEJU TANGO125 COLORIS...
16,https://www.fulloccaz.com/fiches/index/2044815...,2044815,2021/06/06 - 14h16,04/06/2021,KAWASAKI Z650,Roadster,Noir,5590.0,CANNES,6400,JET 7 BIKE,Daniel ou Kévin,11/06/2019,7200.0,4T,7 CV,Constructeur jusqu'au 11/06/2021,First hand,NaN,650.0,Description VÉHICULE POUR PERMIS A2 ET EN BO...
17,https://www.fulloccaz.com/fiches/index/2041644...,2041644,2021/06/06 - 14h16,21/05/2021,KYMCO X.TOWN 125 CBS / GARANTIE 5 ANS,Scooter,Noir,3399.0,PARIS,75017,MFC 17,Jérôme,03/12/2020,1.0,4T,15 CV,Constructeur jusqu'au 23/03/2026,NaN,NaN,125.0,Description KYMCO X-Town 125i CBS Immatricul...
18,https://www.fulloccaz.com/fiches/index/2045773...,2045773,2021/06/06 - 14h16,24/04/2021,TRIUMPH BOBBER 1200,Custom,Bordeaux,10590.0,LANESTER,56600,TRIUMPH LANESTER,Daniel,25/04/2017,11671.0,4T,77 CV,12 mois,First hand,NaN,1200.0,Description TRES BELLE PREMIERE MAIN GARANTI...
19,https://www.fulloccaz.com/fiches/index/2047501...,2047501,2021/06/06 - 14h16,03/06/2021,BMW R 1150 R,Routière,Noir,6699.0,BELFORT,90000,MOTO 90,Claude,01/03/2001,15815.0,4T,NaN,3 mois,NaN,NaN,1150.0,Description BMW R 1150 R pack chrome livrée ...
20,https://www.fulloccaz.com/fiches/index/2015502...,2015502,2021/06/06 - 14h16,29/05/2021,KAWASAKI VERSYS 650 SE GT,Trail,Noir,9590.0,CARCASSONNE,11000,POLE MOTO PASSION CARCASSONNE,"Patrick, Julien, Valentin",04/03/2020,600.0,4T,NaN,Constructeur jusqu'au 04/03/2022,First hand,NaN,650.0,Description VERSYS 650 SE GT PACK GRAND TOUR...
21,https://www.fulloccaz.com/fiches/index/1876832...,1876832,2021/06/06 - 14h16,22/05/2021,KYMCO NEW LIKE 125 MODELE 2019 MOTORS 2019,Scooter,Blanc,2500.0,TOUL,54200,MOTORS 54,NICOLAS,22/05/2021,10.0,4T,1 CV,Constructeur jusqu'au 29/08/2022,NaN,NaN,125.0,Description KYMCO NEW LIKE 125 CC BLANC DISP...
22,https://www.fulloccaz.com/fiches/index/2016872...,2016872,2021/06/06 - 14h16,07/05/2021,PIAGGIO XEVO 125 2009,Scooter,Argent,1899.0,RODEZ,12000,CENTRE DEUX ROUES RODEZ,Service commercial Service commercial,15/09/2009,15000.0,4T,NaN,3 mois,NaN,NaN,125.0,Description Piaggio 125 Xevo 2009Scooter d'o...


In [5]:
data.shape

(2719, 17)